# Lightweight Fine-Tuning Project

TODO: In this cell, describe your choices for each of the following

* PEFT technique: **LoRA**
* Model: **google-bert/bert-base-cased**
* Evaluation approach: **Using accuracy metric**
* Fine-tuning dataset: **stanfordnlp/imdb**

## Loading and Evaluating a Foundation Model

TODO: In the cells below, load your chosen pre-trained Hugging Face model and evaluate its performance prior to fine-tuning. This step includes loading an appropriate tokenizer and dataset.

In [1]:
! pip install -U accelerate -q
! pip install -U transformers -q


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


In [2]:
!pip install datasets evaluate scikit-learn -q


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


In [18]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [19]:
print(device)

cpu


## Prepare the Foundation Model

### Load a pretrained HF model

In [1]:
from transformers import AutoTokenizer, GPT2Tokenizer
model_id="openai-community/gpt2"
tokenizer = GPT2Tokenizer.from_pretrained(model_id, pad_token = '[PAD]')
tokenizer.padding_side = "right" 
# tokenizer.pad_token = '[PAD]' #tokenizer.eos_token # to avoid an error
# tokenizer.pad_token = GPT2Tokenizer.eos_token # to avoid an error

# GPT2_tokenizer.pad_token = GPT2_tokenizer.eos_token

In [2]:
print(tokenizer.pad_token)

[PAD]


### Load and preprocess a dataset

In [3]:
from datasets import load_dataset
dataset = load_dataset("stanfordnlp/imdb")

In [4]:
dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    unsupervised: Dataset({
        features: ['text', 'label'],
        num_rows: 50000
    })
})

In [5]:
def tokenize_function(examples):
    # return tokenizer(examples["text"], truncation=True, padding="max_length") #no padding for GPT
    return tokenizer(examples["text"], padding="max_length", truncation=True, return_tensors='pt') #no padding for GPT

tokenized_train_datasets = dataset["train"].map(tokenize_function, batched=True)
tokenized_test_datasets = dataset["test"].map(tokenize_function, batched=True)

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

In [6]:
small_train_dataset = tokenized_train_datasets.shuffle(seed=42).select(range(3000))
small_eval_dataset = tokenized_test_datasets.shuffle(seed=42).select(range(1000))

In [7]:
print(small_eval_dataset)

Dataset({
    features: ['text', 'label', 'input_ids', 'attention_mask'],
    num_rows: 1000
})


In [8]:
#From: https://achimoraites.medium.com/lightweight-roberta-sequence-classification-fine-tuning-with-lora-using-the-hugging-face-peft-8dd9edf99d19

# from transformers import DataCollatorWithPadding

# # Extract the number of classess and their names
# num_labels = dataset['train'].features['label'].num_classes
# class_names = dataset["train"].features["label"].names
# print(f"number of labels: {num_labels}")
# print(f"the labels: {class_names}")

# # Create an id2label mapping
# # We will need this for our inference.
# id2label = {i: label for i, label in enumerate(class_names)}
# print("id2label=", id2label)
# data_collator = DataCollatorWithPadding(tokenizer=tokenizer, return_tensors="pt")

### Evaluate the pretrained model

In [9]:
# !pip install torch
!pip3 install --pre torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/nightly/cpu

Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/nightly/cpu

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


In [10]:
import torch
from transformers import GPT2ForSequenceClassification #AutoModelForSequenceClassification

model = GPT2ForSequenceClassification.from_pretrained(
    model_id, 
    num_labels=2,
    id2label={0: "negative", 1: "positive"},
    label2id={"negative": 0, "positive": 1}
)

Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at openai-community/gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [11]:
print(model)

GPT2ForSequenceClassification(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2SdpaAttention(
          (c_attn): Conv1D(nf=2304, nx=768)
          (c_proj): Conv1D(nf=768, nx=768)
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D(nf=3072, nx=768)
          (c_proj): Conv1D(nf=768, nx=3072)
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (score): Linear(in_features=768, out_features=2, bias=False)
)


In [12]:
# Print a dataset sample
import random


# Generate a random integer within the range
x = random.randint(0, 1000)

print("text: {},\nlabel:{}".format(small_eval_dataset["text"][x], small_eval_dataset["label"][x]) )

text: Gritty drama? Emotionally powerful? Blah! The BBC has lost out big time to masterful productions such as The Wire, Sopranos and Carnivale from HBO. Okay, so the budget may be a lot smaller but 'The Street' last night was badly acted, predictable, unrealistic, stereotypical, insensitive and a big fat waste of time. TV (British TV) is not as good as it used to be and is falling further and further behind the American productions.<br /><br />There was no sense of brutal violence from the 'local gangster'. There was no indication that this man was 'insane' enough to beat up a man he has respected for such a long time. There was no remorse when he did it and this shouldn't be the sort of character that would back down when Bob Hoskins called his son a pansy, in a display of 'bravery'.<br /><br />I wish I was more eloquent to express my disdain for this show, but I am not and although I can't prove my point well enough, believe me when I say that this was rubbish, shock TV, that provid

In [13]:
#Use accuracy metric
#Function inspired from https://huggingface.co/learn/nlp-course/en/chapter3/3#evaluation
import numpy as np

import evaluate

metric = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)


In [14]:
!pip list

Package                   Version
------------------------- ------------------
accelerate                1.2.1
aiohappyeyeballs          2.4.4
aiohttp                   3.11.11
aiosignal                 1.3.2
anyio                     4.7.0
appnope                   0.1.4
argon2-cffi               23.1.0
argon2-cffi-bindings      21.2.0
arrow                     1.3.0
asttokens                 3.0.0
async-lru                 2.0.4
attrs                     24.3.0
babel                     2.16.0
beautifulsoup4            4.12.3
bleach                    6.2.0
certifi                   2024.12.14
cffi                      1.17.1
charset-normalizer        3.4.0
comm                      0.2.2
datasets                  3.2.0
debugpy                   1.8.11
decorator                 5.1.1
defusedxml                0.7.1
dill                      0.3.8
evaluate                  0.4.3
executing                 2.1.0
fastjsonschema            2.21.1
filelock                  3.16.1
fqdn     

In [15]:
from transformers import TrainingArguments

# training_args = TrainingArguments(output_dir="model_evaluation")
args = TrainingArguments(
    "evaluate_foundational_model_gpt2") 
    # evaluation_strategy="epoch",
    # per_device_train_batch_size=32,
    # per_device_eval_batch_size=32)

training_args = args.set_dataloader(train_batch_size=16, eval_batch_size=64)

In [16]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=small_train_dataset,
    eval_dataset=small_eval_dataset,
    compute_metrics=compute_metrics,
    tokenizer=tokenizer,
)


/var/folders/sj/nx7dbc892dq643lz26s776xr0000gn/T/ipykernel_4041/1996802471.py:3: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [17]:
%%time
import numpy as np

# Let's see the perfomance of the foundation model before any prior training
trainer.evaluate(eval_dataset=small_eval_dataset)

AssertionError: Cannot handle batch sizes > 1 if no padding token is defined.

## **Without any fine tuning the model "google-bert/bert-base-cased" has an _accuracy_ of _0.488_**

### Saving the foundation model to local directory

In [2]:
# Save the foundational model to the local directory "foundational_model/" 
trainer.save_model("foundational_model/")

NameError: name 'trainer' is not defined

## Performing Parameter-Efficient Fine-Tuning

TODO: In the cells below, create a PEFT model from your loaded model, run a training loop, and save the PEFT model weights.

Create two PEFT models to test two different lora_config values and compare the results between the two

### PEFT model (Same foundational model for the two PEFT configuraiotns)

In [ ]:
peft_model_id = model_id 
model = AutoModelForSequenceClassification.from_pretrained(
    peft_model_id,
    num_labels=2,
    id2label={0: "negative", 1: "positive"},
    label2id={"negative": 0, "positive": 1}
)

### Create a PEFT model #1

In [ ]:
# Create an dictiopnary with two set of values for two training to see the impact on the performance of the model
peft_values= {
    "values1": {
        "r": 16,
        "lora_alpha": 16,
        "lora_dropout": 0.1,
        "bias": "none"
    },
    "values2": {
        "r": 64,
        "lora_alpha": 128,
        "lora_dropout": 0.05,
        "bias": "none"
    }
}

In [ ]:
from peft import LoraConfig, TaskType

lora_config1 = LoraConfig(
    task_type=TaskType.TOKEN_CLS,
    r=peft_values["values1"]["r"],
    lora_alpha=peft_values["values1"]["lora_alpha"],
    lora_dropout=peft_values["values1"]["lora_dropout"],
    bias=peft_values["values1"]["bias"],
    target_modules=["query", "value"]
)

In [ ]:
from peft import get_peft_model

lora_model1 = get_peft_model(model, lora_config1)
lora_model1.print_trainable_parameters()

### Train the PEFT model #1

In [ ]:
training_args_peft1 = TrainingArguments(
    "trainer_peft1_output",
    evaluation_strategy="epoch",
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16)

In [ ]:
%%time
trainer1 = Trainer(
    model=lora_model1,
    args=training_args_peft1,
    train_dataset=small_train_dataset,
    eval_dataset=small_eval_dataset,
    compute_metrics=compute_metrics,
)
trainer1.train()

In [ ]:
%%time
trainer1.evaluate()

###### **With fine tuning the model1 "google-bert/bert-base-cased" the _accuracy_ is now _0.849_ much better than the performance of the original foundational model.**

### Save the PEFT model #1

In [ ]:
lora_model1.save_pretrained("trainer_peft_1")

In [ ]:
!ls -ltra trainer_peft_1/

### Create PEFT model #2

In [ ]:
from peft import LoraConfig, TaskType

lora_config2 = LoraConfig(
    task_type=TaskType.TOKEN_CLS,
    r=peft_values["values2"]["r"],
    lora_alpha=peft_values["values2"]["lora_alpha"],
    lora_dropout=peft_values["values2"]["lora_dropout"],
    bias=peft_values["values2"]["bias"],
    target_modules=["query", "value"]
)

In [ ]:
from peft import get_peft_model

lora_model2 = get_peft_model(model, lora_config2)
lora_model2.print_trainable_parameters()

### Train PEFT model #2

In [ ]:
training_args_peft2 = TrainingArguments(
    "trainer_peft2_output",
    evaluation_strategy="epoch",
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16)

In [ ]:
%%time
trainer2 = Trainer(
    model=lora_model2,
    args=training_args_peft2,
    train_dataset=small_train_dataset,
    eval_dataset=small_eval_dataset,
    compute_metrics=compute_metrics
)
trainer2.train()

In [ ]:
%%time
trainer2.evaluate()

**With fine tuning the model2 "google-bert/bert-base-cased" the _accuracy_ is now _0.877_ much better than the performance of the original foundational model and the PEFT1 model.**

### Save the PEFT model #2

In [ ]:
lora_model1.save_pretrained("trainer_peft_2")

In [ ]:
!ls -ltra trainer_peft_2/

## Performing Inference with a PEFT Model

TODO: In the cells below, load the saved PEFT model weights and evaluate the performance of the trained PEFT model. Be sure to compare the results to the results from prior to fine-tuning.

## Perform Inference Using the Fine-Tuned Model

### Load the saved PEFT model

We load the best PEFT model of the two we created: "trainer_peft_2"

In [ ]:
saved_model = AutoModelForSequenceClassification.from_pretrained("trainer_peft_2")


### Evaluate the fine-tuned model

In [ ]:
%%time

# classify function from URL: https://achimoraites.medium.com/lightweight-roberta-sequence-classification-fine-tuning-with-lora-using-the-hugging-face-peft-8dd9edf99d19

x = random.randint(0, 1000)

text_to_classify=small_eval_dataset["text"][x]

def classify(text):
  inputs = tokenizer(text, truncation=True, padding=True, return_tensors="pt")
  output = saved_model(**inputs)

  prediction = output.logits.argmax(dim=-1).item()

  print(f'\n Class: {prediction}, Label: {id2label[prediction]},\nText: {text}')



classify(text_to_classify)

print("\nFrom the dataset, the text is classified as: {}: {}\n".format(small_eval_dataset["label"][x], id2label[small_eval_dataset["label"][x]]))

**The inference classified the text as negative which matches what the datatsets has as label.**